### Imports for all parts

In [ ]:
import numpy as np
import sympy as sym
from os import close
from mpmath.visualization import plot
import numpy as np
from sympy.plotting.plot import plot_parametric
from sympy.solvers import solve, solveset
from sympy import Symbol, lambdify
import matplotlib.pyplot as plt
from sympy.plotting import plot3d, plot
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from sympy import *
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
import matplotlib.cm as cm
import matplotlib.transforms as mtrans

from sklearn.neighbors import NearestCentroid
from sklearn.metrics import (confusion_matrix, plot_roc_curve,
                             plot_confusion_matrix, roc_curve, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances



# Q1

In [ ]:
# Dataset definition

X = [
    [-10, -10, -8, -5, -4, -2, 0, 2, 3, 4, 5, 5, 8],
    [-6, -5, -3, -2, -3, 1, 1, 4, 4, 3, 3, 5, 9]
]

labels = [0] * 7 + [1] * 6

Y = np.array(labels)
X = np.array(X)


Lambda = np.array (
    [
     [1, 10],
     [3, 1]
    ]
)

p = np.array([7, 6])
# p = p / p.sum()

# Computing mean and variance for each class
x_red = X[...,Y==1]
x_yellow = X[...,Y==0]


mr = x_red.mean(axis=1, keepdims=True)
sigma_r = np.cov(x_red)

my = x_yellow.mean(axis=1, keepdims=True)
sigma_y = np.cov(x_yellow)


print(f"mean_red = \n {mr} \n Sigma_red = \n{sigma_r}")
print(f"mean_yellow = \n{my} \n Sigma_yellow = \n{sigma_y}")


# Solving the equation for the border
x1 = Symbol('x')
x2 = Symbol('y')
x = np.array([x1, x2]).reshape(-1, 1)


fy = -1/2*(x-my).T @ np.linalg.inv(sigma_y) @ (x-my)
fr = -1/2*(x-mr).T @ np.linalg.inv(sigma_r) @ (x-mr)

fy, fr = fy[0,0], fr[0,0]


const = np.log(p[0]/p[1] * (Lambda[0,0] - Lambda [1, 0]) / (Lambda[1, 1] - Lambda[0,1]) \
              * np.sqrt(np.linalg.det(sigma_r) / np.linalg.det(sigma_y)))

# there are 2 solutions to the equation
eq1, eq2 = solve(const - fy + fr, [x1,x2])






# Plotting the decision line

tx1 = 10
tx2 = 15

X1 = np.linspace(X[0,:].min() - tx1, X[0,:].max() + tx1, 100)
X2 = np.linspace(X[1,:].min() - tx2, X[1,:].max() + tx2, 100)


f1 = lambdify([x1, x2], eq1[0])
f2 = lambdify([x1, x2], eq2[0])

V1 = np.array([f1(0, v2) for v2 in X2])
V2 = np.array([f2(0, v2) for v2 in X2])

fig,ax = plt.subplots()

ax.plot(V1, X2, label="eq1", c="purple")
ax.plot(V2, X2, label="eq2", c="navy")
ax.scatter(X[0, :], X[1, :], c=Y)

ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_title("Datapoints and Potential Deicision Lines")

ax.legend()
ax.set_xlim(X[0, :].min()-10, X[0, :].max() + 10)
ax.set_ylim(X[1, :].min()-15, X[1, :].max() + 15)
ax.grid()
plt.show()

# Q3

### Q3.C

In [ ]:
# Solving the third part


l = Symbol('lambda')

x0 = np.array([1.0,1.0], dtype=np.longdouble).reshape(-1, 1)
A = Matrix([
              [1.0, 3.0],
              [1.0, 5.0]
              ])
A = A = Matrix([
              [1.0, 0.0],
              [0.0, 1.0]
              ])


# The matrix that gives us xmin
M = (eye(2) - l/2 * (A + A.T)).inv()

xmin = M @ x0
# the equation which xmin should satisfy
eq = (xmin.T @ A @ xmin)[0,0] - 1.0


ans = solve(eq , l)
ans = [np.array(a, dtype=np.clongdouble) for a in ans]
f1 = lambdify([l], xmin)
for a in ans:
  x = np.array(f1(a), dtype=np.clongdouble).real
  print(a, np.linalg.norm(x - x0), x)


# Q6

### Q6.A

In [ ]:

df = pd.read_csv("Iris.csv", index_col=False)

elements = list(combinations(df.columns[:-1], 2))
nrows = len(elements) // 2
ncols = 2

encoder = LabelEncoder()
df["colors"] = encoder.fit_transform(df["Class"])

cmap = cm.get_cmap("viridis", len(encoder.classes_))

fig, axes = plt.subplots(nrows, ncols, figsize=(9, 12)) 
for subplot_number ,((f1, f2), ax) in enumerate(zip(elements, axes.flat)):
  for idx, class_ in enumerate(encoder.classes_):
    sub_df = df.loc[df["colors"] == idx]
    ax.scatter(sub_df[f1], sub_df[f2], color=cmap(idx), label=class_)
    ax.set_xlabel(f1)
    ax.set_ylabel(f2)
    ax.set_title(f"({subplot_number + 1})")

handles, labels = ax.get_legend_handles_labels()
fig.tight_layout(rect=(0.05, 0.05, 0.95, 0.9), h_pad=2, w_pad=0)
fig.legend(handles, labels, loc = (0.25, 0.92), ncol=3 )
fig.suptitle("Feature Space Visualization (2-Combinations).", fontsize=18)



### Q6.B

In [ ]:
# Splitting data into train set and test set
np.random.seed(42)

X = df[df.columns[:-2]].to_numpy()
y = df["colors"].to_numpy()

indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices, ...]
y = y[indices, ...]
split = (len(df) * 8) // 10

X_train, X_test = X[:split, ...], X[split:, ...]
y_train, y_test = y[:split, ...], y[split:, ...]

In [ ]:
# classifier for this task
class MeanClassifier:
  def __init__(self):
    self.means = None
  
  def fit(self, X, y):
    classes = set(y)
    self.means = {}
    for c in classes:
      indices = y == c
      self.means[c] = X[indices, ...].mean(axis=0)
  
  def predict(self, X):
    y = np.zeros((X.shape[0], len(self.means)))
    for idx, mu in enumerate(self.means.values()):
      y[:, idx] = np.sqrt(np.sum((X - mu) ** 2, axis=1)) # Euclidean distance
    return np.argmin(y, axis=1)


class ConfusionMatrix:
  def __init__(self, n_classes=3):
    self.n_classes = n_classes
  
  def __call__(self, true, pred):
    n_classes = self.n_classes
    mat = np.zeros((n_classes, n_classes))
    indices = np.array([true, pred])
    # each (i, j) can be seen as a coordinate in mat
    for i, j in zip(true, pred):
      mat[i, j] += 1
    return mat


def compute_f1(mat):
  f1 = np.zeros((len(mat),))
  #  true predicitons are on diagonal
  recall = np.diag(mat)/mat.sum(axis=1)
  precision = np.diag(mat)/mat.sum(axis=0)
  f1 = 2 * (recall * precision) / (recall + precision)
  return f1

def compute_acc(mat):
  #  true predicitons are on diagonal
  return np.diag(mat).sum() / mat.sum()

In [ ]:
clf = MeanClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
conf_mat = ConfusionMatrix()(true=y_test, pred=pred)
print(conf_mat, compute_f1(conf_mat).mean(), compute_acc(conf_mat))
print(encoder.classes_)

### Q6.C

In [ ]:

np.random.seed(42)
X = df[df.columns[:-2]].to_numpy()
y = df["colors"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

clf = NearestCentroid()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
labels_pred, labels_test = encoder.inverse_transform(pred), encoder.inverse_transform(y_test)
conf_mat = confusion_matrix(y_true=labels_test, y_pred=labels_pred, labels=encoder.classes_)
fig, ax  = plt.subplots(figsize=(10,10))
plot_confusion_matrix(clf, X_test, y_test, display_labels=encoder.classes_, ax=ax)

# Nearest Centroid doesn't have a predict_proba, so normalized distance is the probability

fig, axes = plt.subplots(3, figsize=(8, 12))
for idx, (ax, c) in enumerate(zip(axes.flat, encoder.classes_)):
  y_curr = (y == idx).astype(np.int)
  proba = euclidean_distances(X, clf.centroids_)
  proba = np.max(proba / np.sum(proba,axis=1).reshape(-1, 1), axis=1)
  
  fpr, tpr, _ = roc_curve(y_curr,  proba)
  auc = roc_auc_score(y_curr, proba)
  ax.plot(fpr,tpr,label="auc="+str(auc))
  ax.set_xlabel("FPR")
  ax.set_ylabel("TPR")
  ax.set_title(f"Class: {c}")

fig.tight_layout(rect=(0.05, 0.05, 0.95, 0.95))
fig.suptitle("Plotting ROC curve for different classes", fontsize=20)





# Q8

In [ ]:
# Reading dataset
df = pd.read_csv("quality_test.csv", index_col=False, header=None)
df.columns = ["f1", "f2", "Class"]


X , y = df[["f1", "f2"]].to_numpy(), df["Class"].to_numpy().astype(np.int)


def transform(X):
  max_pow = 7
  X1 = X[:, 0][...,np.newaxis]
  X2 = X[:, 1][...,np.newaxis]
  l = [X1, X2]
  # Transforming the feature set
  for cur_pow in range(2, max_pow+1):
    for i in range(cur_pow, -1, -1):
      j = cur_pow-i
      l.append((X1**i) * (X2 **j))

  X_transform = np.concatenate(l, axis=1)
  return X_transform

X_transform = transform(X)

In [ ]:
# Logistic Regression 

class LogisticRegression:
  # Activation 
  class Sigmoid:
    def __call__(self, z):
      return 1 / (1 + np.exp(-z))
  # Loss
  class BinaryCrossEntropy:
    def __call__(self, y_true, y_pred):
      return -(y_true.T @ np.log(y_pred + 10e-12) + (1-y_true).T @ np.log(1-y_pred + 10e-12)).sum()
    def der(self, y_true, y_pred):
      return y_pred - y_true

  def __init__(self, epochs = 100, lr=10e-3, l2=10e-5):
    self.activation = LogisticRegression.Sigmoid()
    self.loss_function = LogisticRegression.BinaryCrossEntropy()
    self.epochs = epochs # Number of training epochs
    self.lr = lr # Learning rate
    self.l2 = l2 # L2 regularization rate

  def fit(self, X, y):
    self.W = np.random.normal(scale=0.2, size=(X.shape[1] + 1,1))
    # Need to consider bias
    Xt = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    for e in range(self.epochs):
      pred = self.activation(Xt @ self.W)
      loss = self.loss_function(y_true=y, y_pred=pred) + (self.W ** 2).sum() * self.l2/2
      # Summation of all gradients
      grad = (Xt * self.loss_function.der(y_true=y, y_pred=pred)).sum(axis=0, keepdims=True).T
      self.W -= self.lr * grad 
      self.W -= self.l2 * self.W.sum(keepdims=True)
  
  def predict(self, X):
    Xt = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    return (self.activation(Xt @ self.W) > 0.5).astype(np.int)

  def proba(self, X):
    Xt = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    return self.activation(Xt @ self.W)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size=0.2, shuffle=True, random_state=42)

y = y.reshape(-1, 1)
clf = LogisticRegression()
clf.fit(X_train, y_train)
print("Train Accuracy", (clf.predict(X_train) == y_train).astype(np.int).mean())
print("Test Accuracy", (clf.predict(X_test) == y_test).astype(np.int).mean())

In [ ]:
# Limits of X for plotting
xmin , xmax = X.min(axis=0), X.max(axis=0)
xmin -=  0.5 * abs(xmin)
xmax +=  0.5 * abs(xmax)
x1range = np.arange(xmin[0], xmax[0], (xmax[0] - xmin[0])/100)
x2range = np.arange(xmin[1], xmax[1], (xmax[1] - xmin[1])/100)

# Creating a meshgrid to look at all potential values
xx, yy = np.meshgrid(x1range, x2range)

# Transforming the meshgrid into a feature matrix
XX = np.concatenate((xx.reshape(-1, 1), yy.reshape(-1,1)), axis=1)
XX_transform = transform(XX)

# Predicting the values of meshgrid, class and probability
preds = clf.predict(XX_transform)
proba = clf.proba(XX_transform)
print(preds.shape, XX_transform.shape, xx.shape)


# Plotting
fig, axes = plt.subplots(nrows=2, figsize=(10,10), sharex=True, sharey=True)

ax = axes[0]
ax.contourf(xx, yy, proba.reshape(xx.shape),cmap=cm.get_cmap("bone"))
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm.get_cmap("bwr",2))
ax.set_title("Probability Contours")
ax.set_xlabel("F1")
ax.set_ylabel("F2")

ax = axes[1]
ax.contourf(xx, yy, preds.reshape(xx.shape),levels=10, cmap=cm.get_cmap("bone", 4))
handles = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm.get_cmap("bwr",2))
ax.set_title("Decision Boundary")
ax.set_xlabel("F1")
ax.set_ylabel("F2")

fig.suptitle("Decision Boundary and Probabilty Contours", fontsize=20)
plt.tight_layout(rect=(0.05, 0.05, 0.95, 0.95))